Notebook to run the model using pure python.

Template for the C implementation of the model.

In [1]:
from typing import Iterable, Dict

import numpy as np

In [2]:
weights = {}

In [3]:
for i in range(3):
    base_name = f"weights/conv{i}_{{}}.npy"
    weights[f"conv{i}_kernel"] = np.load(base_name.format("kernel"))
    weights[f"conv{i}_bias"] = np.load(base_name.format("bias"))
    
weights["dense_kernel"] = np.load("weights/dense_kernel.npy")
weights["dense_bias"] = np.load("weights/dense_bias.npy")

In [4]:
def apply_2x2_conv_kernel(x: np.ndarray, kernel: np.ndarray) -> np.ndarray:
    # we only need 2x2, no padding :D
    out_rows = x.shape[0] - 1
    out_cols = x.shape[1] - 1
    out = np.zeros((out_rows, out_cols))
    
    for row in range(out_rows):
        for col in range(out_cols):
            sliding_window = x[row:row+2, col:col+2, :]
            out[row, col] = (sliding_window * kernel).sum()
    return out


def apply_2x2_conv_kernels(x: np.ndarray, kernels: np.ndarray) -> np.ndarray:
    # we only need 2x2, no padding :D
    out_rows = x.shape[0] - 1
    out_cols = x.shape[1] - 1
    out = np.zeros((out_rows, out_cols, kernels.shape[-1]))
    
    for row in range(out_rows):
        for col in range(out_cols):
            sliding_window = x[row:row+2, col:col+2, :]
            out[row, col, :] = (sliding_window[...,np.newaxis] * kernels).sum(axis=(0,1,2))
    return out

            
def apply_full_2x2_conv_layer(x: np.ndarray, kernel: np.ndarray,
                              bias: np.ndarray) -> np.ndarray:
    #convolved = np.stack(
    #    [apply_2x2_conv_kernel(x, kernel[...,i]) 
    #     for i in range(kernel.shape[-1])],
    #    axis=-1
    #)
    convolved = apply_2x2_conv_kernels(x, kernel)
    with_bias = convolved + bias
    
    # apply relu
    return np.maximum(with_bias, 0)


def apply_conv_layer_stack(x: np.ndarray, kernels: Iterable[np.ndarray],
                           biases: Iterable[np.ndarray]) -> np.ndarray:
    for kernel, bias in zip(kernels, biases):
        x = apply_full_2x2_conv_layer(x, kernel, bias)
    return x


def apply_max_pool(x: np.ndarray) -> np.ndarray:
    return np.max(x, axis=(0,1))


def softmax(x: np.ndarray) -> np.ndarray:
    e = np.exp(x)
    return e / e.sum()


def apply_dense_output_layer(x: np.ndarray, kernel: np.ndarray,
                             bias: np.ndarray) -> np.ndarray:
    out = x@kernel + bias
    return softmax(out)


def apply_full_model(x: np.ndarray, weights: Dict[str, np.ndarray]):
    x = apply_conv_layer_stack(
        x,
        [weights[f"conv{i}_kernel"] for i in range(3)],
        [weights[f"conv{i}_bias"] for i in range(3)]
    )
    x = apply_max_pool(x)
    x = apply_dense_output_layer(
        x,
        kernel=weights["dense_kernel"],
        bias=weights["dense_bias"]
    )
    return x

In [5]:
test_input = np.array([
    [1,1,0,0,0],
    [0,0,0,0,0],
    [0,0,1,0,0],
    [0,0,1,1,0],
    [0,0,1,0,1]
    ])[..., np.newaxis]

In [6]:
apply_2x2_conv_kernels(test_input, weights["conv0_kernel"])[0,0,0]

2.939501404762268

In [7]:
apply_2x2_conv_kernels(test_input, weights["conv0_kernel"])[2,1,3]

0.9045984745025635

In [8]:
res = apply_full_2x2_conv_layer(test_input, weights["conv0_kernel"], weights["conv0_bias"])
print(res[0,0,0])
print(res[2,1,3])

2.86903178691864
0.9044170095148729


In [9]:
weights["conv0_kernel"][0,1,0,2]

-1.3210745

In [10]:
weights["conv0_kernel"].flatten()

array([ 1.5657663 ,  0.88373774, -1.2657819 , -0.68050164,  1.3737351 ,
       -1.0296128 , -1.3210745 ,  0.36918926,  0.76399225,  1.5246062 ,
       -0.77700794,  1.9321772 , -0.89375335, -0.785154  ,  0.58246917,
        0.5354092 ], dtype=float32)

In [11]:
weights["conv0_bias"].flatten()

array([-7.0469618e-02,  2.3160256e-01,  6.1115247e-01, -1.8146499e-04],
      dtype=float32)

In [12]:
apply_full_model(
    test_input,
    weights
)

array([7.16413923e-03, 9.63031818e-01, 3.46890334e-03, 1.33830939e-03,
       6.50285776e-03, 2.75586578e-03, 1.04178154e-02, 1.22738426e-03,
       5.06467654e-04, 3.58643953e-03])